In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## CubeViz Example

In [1]:
#filename = '../../../cubeviz/data/manga-7495-12704-LOGCUBE_fixed.fits.gz'
filename = '/opt/data/cubeviz/det_image_ch1-short_s3d.fits'

### Create the VizApp for state information

In [2]:
from jdaviz.vizcomponents.applications import cubeviz
from jdaviz.vizcomponents.vizapp import VizApp
vizapp = VizApp()

/opt/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:996: UserWarning: Illegal line #1
	"import matplotlib 
"
	in file "/home/craig/.config/matplotlib/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)
/opt/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:996: UserWarning: Illegal line #2
	"matplotlib.use('Agg')
"
	in file "/home/craig/.config/matplotlib/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)


### Create CubeViz and Show

In [3]:
c = cubeviz.CubeViz(filename, vizapp)

/opt/miniconda3/lib/python3.7/site-packages/glue/core/data_factories/fits.py:155: UserWarning: Dropping column 'ASDF_METADATA' since it is not 1-dimensional
  warnings.warn("Dropping column '{0}' since it is not 1-dimensional".format(column_name))
/opt/miniconda3/lib/python3.7/site-packages/glue/viewers/common/viewer.py:179: UserWarning: Add large data set?
  warnings.warn(message)


In [4]:
c.show()

Box(children=(VBox(children=(HBox(box_style='success', children=(Dropdown(layout=Layout(width='10em'), options…